# 풀 스케일 시뮬레이션

이전까지 페르소나를 생성하고 소규모, 중간규모 시뮬레이션을 통해 안정적으로 예측하고 있는 것을 확인했다.
이번에는 완전한 스케일로 시뮬레이션 해보겠다.

## 📋 시스템 특징
__✅ 주요 기능__
1. 대규모 페르소나 생성 - 카테고리별 150개 × 4 = 600개
2. SMAPE 최적화 - 조미소스(공격적) → 축산캔(극보수적) 차별화
3. 안전한 배치 처리 - 15개씩 처리, 자동 재시도
4. 실시간 모니터링 - 진행률, 비용, 성공률 추적
5. 중간 저장 - 10배치마다 체크포인트 저장
6. 자동 제출 파일 - sample_submission.csv 형식 생성

__🎯 SMAPE 최적화 전략__
```
python조미소스(40% 비중): 평균 25% 구매확률 (moderate 보수성)
참치(27% 비중): 평균 18% 구매확률 (high 보수성)  
우유류: 평균 12% 구매확률 (very_high 보수성)
축산캔: 평균 8% 구매확률 (extreme 보수성)
```

## 환경 세팅

In [1]:
# 환경 설정
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import json
import time
import random
import openai
from openai import OpenAI
from google.colab import userdata
from datetime import datetime, timedelta
from collections import Counter, defaultdict
from dataclasses import dataclass, asdict
import re
import os
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
# API 키 설정
api_key = userdata.get('openai_api_key')
client = OpenAI(api_key=api_key)

In [4]:
# 결과 저장 경로 설정
result_base_path = '/content/drive/MyDrive/Dongwon/result/full_simulation'
os.makedirs(result_base_path, exist_ok=True)

print("✅ 환경 설정 완료")
print(f"📁 결과 저장 경로: {result_base_path}")

✅ 환경 설정 완료
📁 결과 저장 경로: /content/drive/MyDrive/Dongwon/result/full_simulation


## 데이터 로드

In [5]:
# 데이터 로드
df = pd.read_csv("/content/drive/MyDrive/Dongwon/data/raw/product_info.csv")
submission_template = pd.read_csv("/content/drive/MyDrive/Dongwon/data/raw/sample_submission.csv")

print(f"✅ 데이터 로드 완료")
print(f"📊 제품 수: {len(df)}개")
print(f"📄 제출 템플릿: {submission_template.shape}")

# 카테고리별 제품 확인
category_counts = df['category_level_1'].value_counts()
print(f"\n📈 카테고리별 제품 분포:")
for category, count in category_counts.items():
    print(f"  {category}: {count}개")

✅ 데이터 로드 완료
📊 제품 수: 15개
📄 제출 템플릿: (15, 13)

📈 카테고리별 제품 분포:
  조미소스: 6개
  참치: 4개
  우유류: 3개
  축산캔: 2개


## 페르소나 속성 클래스

In [6]:
@dataclass
class PersonaAttributes:
    age_group: str
    gender: str
    income_level: str
    region: str
    family_type: str
    occupation: str
    shopping_style: str
    health_concern: str
    brand_preference: str
    price_sensitivity: str
    product_category: str = ""
    confidence_level: str = ""
    persona_id: str = ""

    def to_dict(self):
        return asdict(self)

# SMAPE 최적화 전략 설정
SMAPE_STRATEGY = {
    "조미소스": {
        "confidence_level": "high_confidence",
        "conservatism": "moderate",  # 40% 비중으로 공격적 예측
        "target_avg_probability": 25,
        "personas_count": 150
    },
    "참치": {
        "confidence_level": "medium_confidence",
        "conservatism": "high",  # 팬덤 효과 불확실성
        "target_avg_probability": 18,
        "personas_count": 150
    },
    "우유류": {
        "confidence_level": "medium_confidence",
        "conservatism": "very_high",  # 복잡한 변수들
        "target_avg_probability": 12,
        "personas_count": 150
    },
    "축산캔": {
        "confidence_level": "low_confidence",
        "conservatism": "extreme",  # 신개념 제품
        "target_avg_probability": 8,
        "personas_count": 150
    }
}

# 속성 풀 정의
ATTRIBUTE_POOLS = {
    "age_group": {
        "20-29세": {"weight": 25, "purchase_propensity": 0.7},
        "30-39세": {"weight": 30, "purchase_propensity": 0.8},
        "40-49세": {"weight": 25, "purchase_propensity": 0.6},
        "50-59세": {"weight": 15, "purchase_propensity": 0.5},
        "60세이상": {"weight": 5, "purchase_propensity": 0.3}
    },
    "gender": {
        "여성": {"weight": 55, "purchase_propensity": 0.7},
        "남성": {"weight": 45, "purchase_propensity": 0.6}
    },
    "income_level": {
        "200만원미만": {"weight": 15, "purchase_propensity": 0.3},
        "200-400만원": {"weight": 35, "purchase_propensity": 0.5},
        "400-600만원": {"weight": 30, "purchase_propensity": 0.7},
        "600만원이상": {"weight": 20, "purchase_propensity": 0.8}
    },
    "region": {
        "서울": {"weight": 20, "purchase_propensity": 0.8},
        "경기": {"weight": 25, "purchase_propensity": 0.7},
        "부산": {"weight": 8, "purchase_propensity": 0.6},
        "기타광역시": {"weight": 20, "purchase_propensity": 0.6},
        "기타지역": {"weight": 27, "purchase_propensity": 0.5}
    },
    "family_type": {
        "1인가구": {"weight": 30, "purchase_propensity": 0.6},
        "신혼부부": {"weight": 15, "purchase_propensity": 0.8},
        "자녀있는가정": {"weight": 40, "purchase_propensity": 0.7},
        "노부부": {"weight": 15, "purchase_propensity": 0.4}
    },
    "occupation": {
        "사무직": {"weight": 35, "purchase_propensity": 0.7},
        "전문직": {"weight": 15, "purchase_propensity": 0.8},
        "주부": {"weight": 20, "purchase_propensity": 0.6},
        "학생": {"weight": 10, "purchase_propensity": 0.5},
        "자영업": {"weight": 12, "purchase_propensity": 0.6},
        "기타": {"weight": 8, "purchase_propensity": 0.5}
    },
    "shopping_style": {
        "신중형": {"weight": 40, "purchase_propensity": 0.5},
        "충동형": {"weight": 20, "purchase_propensity": 0.8},
        "가성비형": {"weight": 25, "purchase_propensity": 0.6},
        "브랜드형": {"weight": 15, "purchase_propensity": 0.7}
    },
    "health_concern": {
        "높음": {"weight": 30, "purchase_propensity": 0.7},
        "보통": {"weight": 50, "purchase_propensity": 0.6},
        "낮음": {"weight": 20, "purchase_propensity": 0.5}
    },
    "brand_preference": {
        "강함": {"weight": 25, "purchase_propensity": 0.7},
        "보통": {"weight": 50, "purchase_propensity": 0.6},
        "약함": {"weight": 25, "purchase_propensity": 0.6}
    },
    "price_sensitivity": {
        "높음": {"weight": 35, "purchase_propensity": 0.4},
        "보통": {"weight": 45, "purchase_propensity": 0.6},
        "낮음": {"weight": 20, "purchase_propensity": 0.8}
    }
}

print("✅ 전역 설정 완료")

✅ 전역 설정 완료


## 페르소나 생성 함수

In [7]:
def adjust_attribute_weights_for_category(base_pools, category):
    """카테고리별 속성 가중치 조정"""
    adjusted_pools = {}

    # 기본 복사
    for attr_name, attr_pool in base_pools.items():
        adjusted_pools[attr_name] = {}
        for option, values in attr_pool.items():
            adjusted_pools[attr_name][option] = values.copy()

    # 카테고리별 특화 조정
    if category == "조미소스":
        # 기존 고객층, 요리 애호가 중심
        adjusted_pools["age_group"]["40-49세"]["weight"] *= 1.8
        adjusted_pools["age_group"]["50-59세"]["weight"] *= 1.5
        adjusted_pools["gender"]["여성"]["weight"] *= 1.3
        adjusted_pools["occupation"]["주부"]["weight"] *= 2.0
        adjusted_pools["shopping_style"]["신중형"]["weight"] *= 1.5
        adjusted_pools["brand_preference"]["강함"]["weight"] *= 1.8

    elif category == "참치":
        # 안유진 팬덤 + 젊은층
        adjusted_pools["age_group"]["20-29세"]["weight"] *= 2.0
        adjusted_pools["age_group"]["30-39세"]["weight"] *= 1.5
        adjusted_pools["gender"]["여성"]["weight"] *= 1.8
        adjusted_pools["shopping_style"]["브랜드형"]["weight"] *= 2.0
        adjusted_pools["shopping_style"]["충동형"]["weight"] *= 1.5

    elif category == "우유류":
        # 건강 트렌드 민감층
        adjusted_pools["age_group"]["20-29세"]["weight"] *= 1.5
        adjusted_pools["age_group"]["30-39세"]["weight"] *= 1.8
        adjusted_pools["health_concern"]["높음"]["weight"] *= 2.0
        adjusted_pools["region"]["서울"]["weight"] *= 1.5

    elif category == "축산캔":
        # 얼리어답터, 간편식 선호층
        adjusted_pools["age_group"]["20-29세"]["weight"] *= 1.3
        adjusted_pools["age_group"]["30-39세"]["weight"] *= 1.5
        adjusted_pools["family_type"]["1인가구"]["weight"] *= 1.8
        adjusted_pools["occupation"]["사무직"]["weight"] *= 1.5
        adjusted_pools["shopping_style"]["충동형"]["weight"] *= 1.5

    return adjusted_pools

In [8]:
def generate_single_persona(adjusted_pools, category, persona_index):
    """단일 페르소나 생성"""
    persona_attrs = {}
    strategy = SMAPE_STRATEGY[category]

    # 각 속성별 가중치 기반 선택
    for attr_name, attr_pool in adjusted_pools.items():
        choices = list(attr_pool.keys())
        weights = [attr_pool[choice]["weight"] for choice in choices]

        # SMAPE 최적화: 보수성 조정
        if strategy["conservatism"] in ["extreme", "very_high"]:
            # 극보수적: 구매 성향 낮은 속성 강화
            for j, choice in enumerate(choices):
                propensity = attr_pool[choice]["purchase_propensity"]
                if propensity < 0.6:
                    weights[j] *= 1.5

        selected = random.choices(choices, weights=weights)[0]
        persona_attrs[attr_name] = selected

    persona = PersonaAttributes(
        **persona_attrs,
        product_category=category,
        confidence_level=strategy["confidence_level"],
        persona_id=f"full_{category}_{persona_index+1:03d}"
    )

    return persona

In [9]:
def generate_personas_for_category(category, count=150):
    """카테고리별 페르소나 대량 생성"""
    print(f"🎭 {category} 카테고리 페르소나 {count}개 생성 중...")

    # 속성 가중치 조정
    adjusted_pools = adjust_attribute_weights_for_category(ATTRIBUTE_POOLS, category)

    personas = []
    for i in range(count):
        persona = generate_single_persona(adjusted_pools, category, i)
        personas.append(persona)

        # 진행률 표시
        if (i + 1) % 50 == 0:
            print(f"  진행: {i+1}/{count} ({(i+1)/count*100:.1f}%)")

    print(f"✅ {category} 페르소나 생성 완료: {len(personas)}개")
    return personas

In [10]:
def generate_all_personas():
    """전체 카테고리 페르소나 생성"""
    print("👥 전체 카테고리 페르소나 생성 시작...")
    print("="*60)

    all_personas = {}
    total_count = 0

    for category in ["조미소스", "참치", "우유류", "축산캔"]:
        personas_count = SMAPE_STRATEGY[category]["personas_count"]
        personas = generate_personas_for_category(category, personas_count)
        all_personas[category] = personas
        total_count += len(personas)

        print(f"  {category}: {len(personas)}개 생성")

    print(f"\n🎉 전체 페르소나 생성 완료: {total_count}개")

    # 페르소나 저장
    save_personas(all_personas)

    return all_personas

In [11]:
def save_personas(all_personas):
    """페르소나 데이터 저장"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{result_base_path}/personas_{timestamp}.pkl"

    with open(filename, 'wb') as f:
        pickle.dump(all_personas, f)

    print(f"💾 페르소나 저장 완료: {filename}")
    return filename

## LLM 프롬프트 생성 함수

In [12]:
def create_system_message():
    """LLM 시스템 메시지 생성"""
    return """당신은 한국 소비자 행동 분석 전문가입니다.
신제품 시장 수용을 현실적이고 보수적으로 예측합니다.

원칙:
- 신제품은 초기 관망세가 있습니다
- 과도한 낙관론을 지양합니다
- 한국 소비자 특성을 반영합니다
- 계절성과 경쟁을 고려합니다"""

In [13]:
def create_user_message(persona, product_name, category):
    """사용자 메시지 생성"""
    # 보수성 지시사항
    conservatism_map = {
        "moderate": "신중하지만 균형잡힌 관점에서",
        "high": "보수적이고 신중한 관점에서",
        "very_high": "매우 보수적이고 현실적인 관점에서",
        "extreme": "극도로 보수적이고 회의적인 관점에서"
    }

    conservatism = conservatism_map[SMAPE_STRATEGY[category]["conservatism"]]

    return f"""
# 소비자 프로필
연령/성별: {persona.age_group} {persona.gender}
소득/지역: {persona.income_level}, {persona.region}
가족: {persona.family_type}, 직업: {persona.occupation}
성향: {persona.shopping_style}, 건강관심: {persona.health_concern}
브랜드선호: {persona.brand_preference}, 가격민감도: {persona.price_sensitivity}

# 신제품
제품: {product_name}
카테고리: {category}
출시: 2024년 7월

# 예측 요청
{conservatism} 이 소비자의 12개월 구매 패턴을 예측하세요.

JSON 응답:
{{
    "monthly_purchase_probability": [월1~월12],  // 0-100%
    "monthly_purchase_frequency": [월1~월12],   // 0-3회
    "reasoning": "예측 근거",
    "confidence_level": 0.0-1.0
}}
"""

In [14]:
def create_prompt_for_simulation(persona, product_name, category):
    """시뮬레이션용 완전한 프롬프트 생성"""
    system_msg = create_system_message()
    user_msg = create_user_message(persona, product_name, category)

    return {
        "system": system_msg,
        "user": user_msg,
        "metadata": {
            "persona_id": persona.persona_id,
            "product_name": product_name,
            "category": category
        }
    }

## LLM API 호출 함수

In [15]:
def parse_llm_response(content):
    """LLM 응답 파싱"""
    try:
        json_match = re.search(r'\{.*\}', content, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group())
            if validate_response_data(data):
                return {"success": True, "data": data}
        return {"success": False, "error": "JSON 형식 문제"}
    except Exception as e:
        return {"success": False, "error": str(e)}

In [16]:
def validate_response_data(data):
    """응답 데이터 검증"""
    required = ["monthly_purchase_probability", "monthly_purchase_frequency", "reasoning", "confidence_level"]

    # 필수 필드 확인
    if not all(field in data for field in required):
        return False

    # 길이 확인
    if len(data["monthly_purchase_probability"]) != 12 or len(data["monthly_purchase_frequency"]) != 12:
        return False

    # 값 범위 확인
    for prob in data["monthly_purchase_probability"]:
        if not isinstance(prob, (int, float)) or not (0 <= prob <= 100):
            return False

    for freq in data["monthly_purchase_frequency"]:
        if not isinstance(freq, (int, float)) or not (0 <= freq <= 3):
            return False

    return True


In [17]:
def call_llm_single(persona, product_name, category):
    """단일 LLM API 호출"""
    start_time = time.time()

    try:
        # 프롬프트 생성
        prompt_data = create_prompt_for_simulation(persona, product_name, category)

        # API 호출
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": prompt_data["system"]},
                {"role": "user", "content": prompt_data["user"]}
            ],
            temperature=0.7,
            max_tokens=1200,
            timeout=45
        )

        end_time = time.time()
        response_time = end_time - start_time

        content = response.choices[0].message.content
        cost = (response.usage.prompt_tokens * 0.0015 + response.usage.completion_tokens * 0.002) / 1000

        # 응답 파싱
        parsed = parse_llm_response(content)

        if parsed["success"]:
            result = {
                "persona_id": persona.persona_id,
                "product_name": product_name,
                "category": category,
                "response": parsed["data"],
                "api_info": {
                    "response_time": response_time,
                    "cost": cost,
                    "tokens": response.usage.total_tokens
                }
            }
            return {"success": True, "result": result, "cost": cost, "time": response_time}
        else:
            return {"success": False, "error": f"파싱 실패: {parsed['error']}", "cost": cost, "time": response_time}

    except Exception as e:
        end_time = time.time()
        return {"success": False, "error": str(e), "cost": 0, "time": end_time - start_time}

## 배치 처리 및 모니터링 함수

In [18]:
def process_single_batch(batch_data, batch_id):
    """단일 배치 처리"""
    print(f"📦 배치 {batch_id} 처리 시작 ({len(batch_data)}개)")

    batch_results = []
    batch_cost = 0
    batch_time = 0
    batch_failures = []

    for i, (persona, product_name, category) in enumerate(batch_data):
        print(f"  [{i+1}/{len(batch_data)}] {category} - {product_name[:20]}...", end=" ")

        result = call_llm_single(persona, product_name, category)

        if result["success"]:
            batch_results.append(result["result"])
            print("✅")
        else:
            print(f"❌ {result['error'][:30]}...")
            batch_failures.append({
                "persona_id": persona.persona_id,
                "product_name": product_name,
                "category": category,
                "error": result["error"]
            })

        batch_cost += result["cost"]
        batch_time += result["time"]

        # Rate limiting
        time.sleep(0.3)

    print(f"✅ 배치 {batch_id} 완료: {len(batch_results)}개 성공, ${batch_cost:.3f}")

    return {
        "results": batch_results,
        "failures": batch_failures,
        "cost": batch_cost,
        "time": batch_time
    }

In [19]:
def save_checkpoint(all_results, all_failures, total_cost, total_time, checkpoint_name):
    """체크포인트 저장"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    checkpoint_file = f"{result_base_path}/checkpoint_{checkpoint_name}_{timestamp}.pkl"

    checkpoint_data = {
        "results": all_results,
        "failures": all_failures,
        "total_cost": total_cost,
        "total_time": total_time,
        "processed_count": len(all_results) + len(all_failures),
        "success_count": len(all_results),
        "failure_count": len(all_failures),
        "timestamp": timestamp
    }

    with open(checkpoint_file, 'wb') as f:
        pickle.dump(checkpoint_data, f)

    print(f"💾 체크포인트 저장: {checkpoint_file}")
    return checkpoint_file

In [20]:
def display_progress(processed, total, successes, failures, cost, elapsed_time):
    """진행률 표시"""
    success_rate = (successes / max(1, processed)) * 100
    avg_time = elapsed_time / max(1, processed)
    remaining = total - processed
    eta = remaining * avg_time

    print(f"\n📊 진행 상황:")
    print(f"  처리: {processed}/{total} ({processed/total*100:.1f}%)")
    print(f"  성공: {successes} | 실패: {failures} | 성공률: {success_rate:.1f}%")
    print(f"  총 비용: ${cost:.2f} | 평균 응답시간: {avg_time:.1f}초")
    print(f"  예상 완료: {eta/3600:.1f}시간 후")

## 메인 실행 함수

In [21]:
def create_simulation_queue(all_personas):
    """시뮬레이션 작업 대기열 생성"""
    print("📋 시뮬레이션 대기열 생성...")

    queue = []
    for _, product in df.iterrows():
        product_name = product['product_name']
        category = product['category_level_1']

        if category in all_personas:
            personas = all_personas[category]
            for persona in personas:
                queue.append((persona, product_name, category))

    print(f"✅ 시뮬레이션 대기열 생성 완료: {len(queue)}개")
    return queue

In [22]:
def execute_full_scale_simulation(all_personas, batch_size=15):
    """풀 스케일 시뮬레이션 메인 실행"""
    print("🚀 풀 스케일 시뮬레이션 실행!")
    print("="*80)

    # 시뮬레이션 대기열 생성
    queue = create_simulation_queue(all_personas)

    # 예상 비용 및 시간 계산
    estimated_cost = len(queue) * 0.25
    estimated_hours = len(queue) * 3 / 3600
    total_batches = len(queue) // batch_size + (1 if len(queue) % batch_size > 0 else 0)

    print(f"📊 시뮬레이션 개요:")
    print(f"  총 호출: {len(queue):,}회")
    print(f"  예상 비용: ${estimated_cost:.2f}")
    print(f"  예상 시간: {estimated_hours:.1f}시간")
    print(f"  배치 크기: {batch_size}개")
    print(f"  총 배치 수: {total_batches}개")

    # 실행 확인
    confirm = input("\n계속 진행하시겠습니까? (y/n): ")
    if confirm.lower() != 'y':
        print("❌ 시뮬레이션 중단")
        return None

    print("\n🚀 시뮬레이션 시작!")
    start_time = datetime.now()

    # 결과 저장용 변수들
    all_results = []
    all_failures = []
    total_cost = 0
    total_time = 0

    # 배치별 처리
    for batch_id in range(total_batches):
        start_idx = batch_id * batch_size
        end_idx = min(start_idx + batch_size, len(queue))
        batch_data = queue[start_idx:end_idx]

        print(f"\n📦 배치 {batch_id + 1}/{total_batches} 처리 중...")

        batch_result = process_single_batch(batch_data, batch_id + 1)

        # 결과 누적
        all_results.extend(batch_result["results"])
        all_failures.extend(batch_result["failures"])
        total_cost += batch_result["cost"]
        total_time += batch_result["time"]

        # 진행률 표시
        processed = len(all_results) + len(all_failures)
        elapsed = (datetime.now() - start_time).total_seconds()
        display_progress(processed, len(queue), len(all_results), len(all_failures), total_cost, elapsed)

        # 주기적 체크포인트 저장 (10배치마다)
        if (batch_id + 1) % 10 == 0:
            save_checkpoint(all_results, all_failures, total_cost, total_time, f"batch_{batch_id + 1}")

        # 배치 간 휴식 (API 안정성)
        if batch_id < total_batches - 1:
            print("⏸️ 배치 간 휴식 (10초)...")
            time.sleep(10)

    end_time = datetime.now()
    duration = end_time - start_time

    print(f"\n🎉 풀 스케일 시뮬레이션 완료!")
    print(f"="*80)
    print(f"실행 시간: {duration}")
    print(f"성공률: {len(all_results)}/{len(queue)} ({len(all_results)/len(queue)*100:.1f}%)")
    print(f"총 비용: ${total_cost:.2f}")
    print(f"실패 횟수: {len(all_failures)}")

    # 최종 결과 저장
    save_final_results(all_results, all_failures, total_cost, total_time)

    return all_results

## 결과 저장 및 제출 파일 생성 함수들

In [23]:
def save_final_results(all_results, all_failures, total_cost, total_time):
    """최종 결과 저장"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 1. 원본 결과 저장
    results_file = f"{result_base_path}/final_results_{timestamp}.json"
    with open(results_file, 'w', encoding='utf-8') as f:
        json.dump({
            "summary": {
                "total_calls": len(all_results) + len(all_failures),
                "successful_calls": len(all_results),
                "failed_calls": len(all_failures),
                "total_cost": total_cost,
                "total_time": total_time,
                "timestamp": timestamp
            },
            "results": all_results,
            "failed_requests": all_failures
        }, f, ensure_ascii=False, indent=2)

    print(f"💾 최종 결과 저장: {results_file}")

    # 2. 제출 파일 생성
    submission_file = generate_submission_file(all_results, timestamp)
    print(f"📄 제출 파일 생성: {submission_file}")

    return results_file, submission_file

def generate_submission_file(all_results, timestamp):
    """제출 파일 생성"""
    print("📄 제출 파일 생성 중...")

    # 제품별 결과 집계
    product_predictions = {}

    for result in all_results:
        product_name = result["product_name"]
        response = result["response"]

        if product_name not in product_predictions:
            product_predictions[product_name] = {
                "probabilities": [],
                "frequencies": []
            }

        product_predictions[product_name]["probabilities"].append(response["monthly_purchase_probability"])
        product_predictions[product_name]["frequencies"].append(response["monthly_purchase_frequency"])

    # 제출 데이터 생성
    submission_data = []

    for product_name in submission_template['product_name']:
        if product_name in product_predictions:
            # 여러 페르소나 결과의 평균 계산
            probs = np.array(product_predictions[product_name]["probabilities"])
            freqs = np.array(product_predictions[product_name]["frequencies"])

            avg_probs = np.mean(probs, axis=0)
            avg_freqs = np.mean(freqs, axis=0)

            # 최종 수요 계산 (확률 × 빈도 × 100)
            monthly_demands = [(prob/100) * freq * 100 for prob, freq in zip(avg_probs, avg_freqs)]

            # 정수로 반올림
            monthly_demands = [max(0, round(demand)) for demand in monthly_demands]
        else:
            # 예측 데이터가 없는 경우 보수적 추정
            monthly_demands = [5] * 12  # 매우 보수적

        row = {"product_name": product_name}
        for i in range(12):
            row[f"months_since_launch_{i+1}"] = monthly_demands[i]

        submission_data.append(row)

    # 데이터프레임 생성 및 저장
    submission_df = pd.DataFrame(submission_data)
    submission_file = f"{result_base_path}/submission_{timestamp}.csv"
    submission_df.to_csv(submission_file, index=False)

    print(f"✅ 제출 파일 생성 완료")
    print(f"   파일: {submission_file}")
    print(f"   형식: {submission_df.shape}")

    # 간단한 검증
    print(f"\n📊 제출 파일 미리보기:")
    print(submission_df.head())

    return submission_file

## 메인 실행 스크립트

In [24]:
def main():
    """메인 실행 함수"""
    print("🎯 풀 스케일 LLM 시뮬레이션 시스템")
    print("="*80)

    try:
        # 1단계: 페르소나 생성
        print("\n1️⃣ 페르소나 생성 단계")
        all_personas = generate_all_personas()

        # 2단계: 시뮬레이션 실행
        print("\n2️⃣ 시뮬레이션 실행 단계")
        results = execute_full_scale_simulation(all_personas, batch_size=15)

        if results:
            print(f"\n🎉 시뮬레이션 성공적으로 완료!")
            print(f"   총 결과: {len(results)}개")
            print(f"   결과 파일: {result_base_path}")

    except KeyboardInterrupt:
        print(f"\n⚠️ 사용자에 의해 중단됨")
        # 현재까지의 결과 저장 (구현 필요시)

    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        raise

# 자동 실행 부분
print("="*80)
print("🚀 풀 스케일 시뮬레이션 자동 시작")
print("="*80)

# 사용자 확인
start_simulation = input("풀 스케일 시뮬레이션을 시작하시겠습니까? (y/n): ")
if start_simulation.lower() == 'y':
    main()
else:
    print("✋ 시뮬레이션을 시작하려면 위의 main() 함수를 호출하세요.")
    print("   예: main()")

🚀 풀 스케일 시뮬레이션 자동 시작
풀 스케일 시뮬레이션을 시작하시겠습니까? (y/n): y
🎯 풀 스케일 LLM 시뮬레이션 시스템

1️⃣ 페르소나 생성 단계
👥 전체 카테고리 페르소나 생성 시작...
🎭 조미소스 카테고리 페르소나 150개 생성 중...
  진행: 50/150 (33.3%)
  진행: 100/150 (66.7%)
  진행: 150/150 (100.0%)
✅ 조미소스 페르소나 생성 완료: 150개
  조미소스: 150개 생성
🎭 참치 카테고리 페르소나 150개 생성 중...
  진행: 50/150 (33.3%)
  진행: 100/150 (66.7%)
  진행: 150/150 (100.0%)
✅ 참치 페르소나 생성 완료: 150개
  참치: 150개 생성
🎭 우유류 카테고리 페르소나 150개 생성 중...
  진행: 50/150 (33.3%)
  진행: 100/150 (66.7%)
  진행: 150/150 (100.0%)
✅ 우유류 페르소나 생성 완료: 150개
  우유류: 150개 생성
🎭 축산캔 카테고리 페르소나 150개 생성 중...
  진행: 50/150 (33.3%)
  진행: 100/150 (66.7%)
  진행: 150/150 (100.0%)
✅ 축산캔 페르소나 생성 완료: 150개
  축산캔: 150개 생성

🎉 전체 페르소나 생성 완료: 600개
💾 페르소나 저장 완료: /content/drive/MyDrive/Dongwon/result/full_simulation/personas_20250822_182647.pkl

2️⃣ 시뮬레이션 실행 단계
🚀 풀 스케일 시뮬레이션 실행!
📋 시뮬레이션 대기열 생성...
✅ 시뮬레이션 대기열 생성 완료: 2250개
📊 시뮬레이션 개요:
  총 호출: 2,250회
  예상 비용: $562.50
  예상 시간: 1.9시간
  배치 크기: 15개
  총 배치 수: 150개

계속 진행하시겠습니까? (y/n): y

🚀 시뮬레이션 시작!

📦 배치 1/150 처리 중...
📦 배치 1 처리 시작 